In [1]:
import pandas as pd
import numpy as np
import imageio
import matplotlib.pyplot as plt
from tensorflow.keras.backend import expand_dims
from tensorflow.keras.preprocessing import image_dataset_from_directory

In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras import Sequential, layers
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf
import joblib

In [3]:
datagen = ImageDataGenerator(
    rescale = 1./255, # need to remember we have normalised here so don't do it within neural networks
    rotation_range = 5,
    zoom_range = (0.90, 0.90),
    brightness_range = (0.95, 0.95),
    horizontal_flip = True,
    vertical_flip = True,
    data_format = 'channels_last',
    validation_split = 0.2,
    dtype = tf.bfloat16
)

In [6]:
train_generator = datagen.flow_from_directory(
    '/Users/LG/code/katarzyna-kupczyk/alzheimers_final_project/raw_data/AlzheimersDataset/test',
    target_size = (208, 176),
    batch_size = (32),
    class_mode = 'categorical',
    shuffle = True,
    subset = 'training',
    seed = 123
)

Found 1025 images belonging to 4 classes.


In [7]:
validation_generator = datagen.flow_from_directory(
    '/Users/LG/code/katarzyna-kupczyk/alzheimers_final_project/raw_data/AlzheimersDataset/test',
    target_size = (208, 176),
    batch_size = (32),
    class_mode = 'categorical',
    shuffle = True,
    subset = 'validation',
    seed = 123
)


Found 254 images belonging to 4 classes.


In [10]:
model = Sequential()
model.add(layers.Conv2D(16, (3,3), input_shape=(208, 176, 3), padding='same', activation="relu"))
model.add(layers.MaxPool2D(pool_size=(2,2)))
model.add(layers.Conv2D(32, (2,2), padding='same', activation="relu"))
model.add(layers.MaxPool2D(pool_size=(2,2))) 
model.add(layers.Flatten())
model.add(layers.Dense(50, activation='relu'))
model.add(layers.Dense(4, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['Accuracy', 'Precision','AUC'])
es = EarlyStopping(patience=5, restore_best_weights=True)
model.fit(train_generator, validation_data=validation_generator, epochs=50, callbacks=[es], verbose=1)

Epoch 1/50
33/33 [==============================] - 27s 797ms/step - loss: 1.1064 - Accuracy: 0.4595 - precision: 0.5233 - auc: 0.7664 - val_loss: 0.9656 - val_Accuracy: 0.5079 - val_precision: 0.6870 - val_auc: 0.8279
Epoch 2/50
33/33 [==============================] - 24s 716ms/step - loss: 0.9940 - Accuracy: 0.5210 - precision: 0.6168 - auc: 0.8139 - val_loss: 0.9561 - val_Accuracy: 0.5236 - val_precision: 0.5966 - val_auc: 0.8283
Epoch 3/50
33/33 [==============================] - 25s 754ms/step - loss: 0.9710 - Accuracy: 0.5278 - precision: 0.6141 - auc: 0.8205 - val_loss: 0.9300 - val_Accuracy: 0.5079 - val_precision: 0.6387 - val_auc: 0.8350
Epoch 4/50
33/33 [==============================] - 22s 666ms/step - loss: 0.9383 - Accuracy: 0.5463 - precision: 0.6408 - auc: 0.8330 - val_loss: 0.9219 - val_Accuracy: 0.5276 - val_precision: 0.6333 - val_auc: 0.8377
Epoch 5/50
33/33 [==============================] - 22s 669ms/step - loss: 0.9125 - Accuracy: 0.5776 - precision: 0.6858 - a

In [9]:
model.evaluate(validation_generator)

NameError: name 'validation_generator' is not defined

In [11]:
joblib.dump(model, '../bestmodel.joblib')

TypeError: cannot pickle 'weakref' object

In [ ]:
def preprocessing(img_path):
    img = Image.open('/Users/LG/code/katarzyna-kupczyk/alzheimers_final_project/raw_data/AlzheimersDataset/train/MildDemented/mildDem54.jpg').convert('L').resize((224, 224), Image.ANTIALIAS)
    img = np.array(img)
    img_preproc = img/255
    return img_preproc

def predict(img_preproc, model_path):
    model = joblib.load(model_path)
    prediction = model.predict(img_preproc)
    return prediction